# Installing Spotipy

In [ ]:
!conda install -c conda-forge spotipy

## Loading credentials from the config file

#### Make sure that you have stored your spotify client_id and client_secret in a separate config.py file
#### Once that's done, we import it

In [ ]:
client_id = 'xxx'
client_secret = 'yyy'
redirect_uri = 'http://localhost:8888/callback'

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
))

track = sp.track('3n3Ppam7vgaVa1iaRUc9Lp')
print(track['name'], 'von', track['artists'][0]['name'])


In [ ]:
import pkg_resources

spotipy_version = pkg_resources.get_distribution("spotipy").version
print(spotipy_version)


## Starting with Spotify API

In [ ]:
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyOAuth

#Spotify-Objekt mit Authentifizierung
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope=["user-library-read", "user-read-playback-state", "playlist-read-private"]
))

# Holen der URL zur Authentifizierung (Umleitung)
auth_url = sp.auth_manager.get_authorize_url()
print("Öffne diese URL im Browser und melde dich bei Spotify an:", auth_url)

# Warte auf den eingegebenen Code
response = input("Füge den Code aus der URL hier ein: ")

# Den Code gegen ein Zugriffstoken eintauschen
token_info = sp.auth_manager.get_access_token(response)
sp = spotipy.Spotify(auth=token_info['access_token'])

# Benutzerinformationen anzeigen
user_info = sp.current_user()
print(user_info)


In [ ]:
import pprint

# Understanding the json

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

results = sp.search(q="2024 Top 100 Germany", type="playlist", limit=10, )

print(json.dumps(results, indent=2, ensure_ascii=False))

## Other Info

## Getting the track_id

In [ ]:
# Suche nach einem Song (z.B. "Shape of You" von Ed Sheeran)
track_name = "Bucket Listener"
result = sp.search(q=track_name, type='track', limit=1)  # Wir suchen nach einem Track

# Extrahiere die Track-ID
track_id = result['tracks']['items'][0]['id']
artist_name = result['tracks']['items'][0]['artists'][0]['name']
print(f"Track-ID of '{track_name}' by '{artist_name}': {track_id}")

## Embedded track player

In [ ]:
from IPython.display import HTML, display

# Funktion zum Einbetten eines Spotify Tracks
def embed_spotify_track(track_id):
    embed_code = f'<iframe src="https://open.spotify.com/embed/track/{track_id}" width="300" height="80" frameBorder="0" allowtransparency="true" allow="encrypted-media"></iframe>'
    display(HTML(embed_code))

# Beispiel: Funktion aufrufen, um den Track einzubetten
embed_spotify_track('2fkRso7UhJxinCamVHv3dC')


# Getting the Audio features of a song

## Building a Data frame of audio features

In [ ]:
# feature is outdated, not existig anymore for the api

## Searching a playlist

In [ ]:
# 1. Suche nach Playlists
def search_playlists(query, limit=10):
    results = sp.search(q=query, limit=limit, type='playlist')
    playlists = results['playlists']['items']
    return playlists

# 2. Zeige die Playlists an
def display_playlists(playlists):
    for idx, playlist in enumerate(playlists):
        print(f"{idx + 1}. {playlist['name']} by {playlist['owner']['display_name']}")
        print(f"Playlist-ID: {playlist['id']}")
        print(f"Link: {playlist['external_urls']['spotify']}")
        print(f"Description: {playlist['description']}\n")

# Beispiel-Suchbegriff (z.B. 'chill music')
search_query = 'late night chill'  # Suchbegriff für Playlists
playlists = search_playlists(search_query)

# 3. Anzeigen der gefundenen Playlists
display_playlists(playlists)


## Extracting a song from playlist

In [ ]:
def get_first_song_from_playlist(playlist_id, limit=10):
    results = sp.playlist_tracks(playlist_id, limit=limit)
    items = results['items']  # <-- hier korrekter Name
    tracks = []
    
    for item in items:
        track = item['track']
        if track:  # manchmal kann das 'None' sein, z.B. wenn ein Song gelöscht wurde
            song_name = track['name']
            artist_name = track['artists'][0]['name']
            uri = track['uri']
            tracks.append({
                'name': song_name,
                'artist': artist_name,
                'uri': uri
            })
    return tracks

playlist_id = "1mUrePLiLRYpJsDekvTcLg"

songs = get_first_song_from_playlist(playlist_id)

for idx, song in enumerate(songs, 1):
    print(f"{idx}. {song['name']} – {song['artist']} (URI: {song['uri']})")


## Extracting the songs of a playlist

Pagination using "next".
When you collect songs from a playlist using sp.playlist_tracks, you're limited by the limit parameter, which has a maximum (and default) value of 100. When the playlist has more than 100 songs, you have to collect them by navigating through the "pages" of the results.

The parameter offset allows you to retrieve resuls starting at a certain position: if you start at position 101, you'd get the next "page" of results. An offset of 201 would give you the third page, and so on.

The function sp.next() does the same, but in a simpler way: it can be used on the results from any request to directly retrieve the results for the next page.

We can check whether there's a next page or not by accessing the key next on the results from any request.

In [ ]:
def extract_song_from_playlist(playlist_id, song_name):
    results = sp.playlist_tracks(playlist_id, limit=100)
    items = results['items']
    
    for item in items:
        track = item['track']
        if track and song_name.lower() in track['name'].lower():
            return {
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'uri': track['uri']
            }
    
    return None  # Falls nichts gefunden wurde


playlist_id = '1mUrePLiLRYpJsDekvTcLg'
search_title = "Chasing Highs"

song = extract_song_from_playlist(playlist_id, search_title)

if song:
    print(f"✅ Gefunden: {song['name']} – {song['artist']}")
    print(f"URI: {song['uri']}")
else:
    print("❌ Song nicht in der Playlist gefunden.")


### Chopping a big playlist into chunks

In [ ]:
def get_all_tracks_in_chunks(playlist_id, chunk_size=100):
    tracks = []
    offset = 0
    total = None

    while True:
        response = sp.playlist_tracks(playlist_id, limit=chunk_size, offset=offset)
        
        if total is None:
            total = response['total']
        
        items = response['items']
        if not items:
            break

        tracks.extend(items)
        offset += chunk_size

        print(f"✅ Chunk geladen: {offset}/{total} Songs")

        if offset >= total:
            break

    return tracks


playlist_id = '1mUrePLiLRYpJsDekvTcLg'
tracks = get_all_tracks_in_chunks(playlist_id)

print(f"🎧 Gesamtzahl Tracks: {len(tracks)}")

# Beispiel: Namen der ersten 5 Tracks
for t in tracks[:5]:
    print(t['track']['name'], "-", t['track']['artists'][0]['name'])



### Example with rock

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Funktion, um die ersten 10 Songs und ihre Genres aus einer Playlist zu holen
def get_first_10_songs_with_genre(playlist_id):
    # Lade die ersten 10 Tracks aus der Playlist
    response = sp.playlist_tracks(playlist_id, limit=10)
    items = response['items']
    
    song_info = []

    for item in items:
        track = item['track']
        if track:
            # Holen des Künstlers und seines Genres
            artist = track['artists'][0]
            artist_id = artist['id']
            artist_info = sp.artist(artist_id)
            genres = artist_info.get('genres', [])
            
            song_info.append({
                'name': track['name'],
                'artist': artist['name'],
                'genres': genres
            })
    
    return song_info

# Beispiel-Playlist-ID
playlist_id = '1mUrePLiLRYpJsDekvTcLg'
songs_with_genres = get_first_10_songs_with_genre(playlist_id)

# Ausgabe der Song-Informationen und der Genres
for idx, song in enumerate(songs_with_genres, 1):
    print(f"{idx}. {song['name']} – {song['artist']}")
    print(f"   Genres: {', '.join(song['genres']) if song['genres'] else 'Keine Genres gefunden'}\n")


1. I Do – Beatsteaks
   Genres: Keine Genres gefunden

2. Mindestens in 1000 Jahren – Frittenbude
   Genres: german pop

3. Chasing Highs – ALMA
   Genres: Keine Genres gefunden

4. Hurra die Welt geht unter - Live – AnnenMayKantereit
   Genres: german pop

5. Yes Sir – Genetikk
   Genres: german hip hop

6. Teenage Dirtbag – Wheatus
   Genres: Keine Genres gefunden

7. Schafe – Herr von Grau
   Genres: german hip hop

8. Sleeping in (Acoustic) – Phil Good
   Genres: Keine Genres gefunden

9. Growing Up – Phil Good
   Genres: Keine Genres gefunden

10. Still Standing – Hilltop Hoods
   Genres: Keine Genres gefunden



## Optional(Extra)

## Getting the artists of the playlist 

# Getting albums 

In this section we will work with albums to extract information. We will start by extracting all the albums of an artist.

Example: Coldplay

## Getting the songs of a given album

In [ ]:
# credentials lösche bevor ich final speicher!!
